In [100]:
import numpy as np
import tensorflow as tf

In [273]:
#Input shape :  4D Tensor (batch , heigth , width , channeLs)
#Output shape: 5D Tensor (batch , height , width , n_boxes , 8)

#Последнияя ось содержит 4 координаты bbox - (cx,cy,w,h) + 4 значения вариации для каждой коробки

class AnchorBoxes:
    
    def __init__(self,img_height=300,
                 img_width=300,
                 this_scale=scales[0],
                 next_scale=scales[1],
                 aspect_ratios=[0.5,1.0,2.0],
                 two_boxes_for_ar1=True,
                 this_steps=None,
                 this_offsets=None,
                 clip_boxes=False,
                 variances=[0.1,0.1,0.2,0.2],
                 coords='centroids',
                 n_boxes=4,#!!!!!!!!!!!!!!!! Убрать это должно рассчитыаваться автоматически после __init__
                 normalize_coords=False):
    
        self.img_width=img_width
        self.img_height=img_height
        self.this_scale=scales[0]
        self.next_scale=scales[1]
        self.aspect_ratios=aspect_ratios
        self.two_boxes_for_ar1=two_boxes_for_ar1
        self.this_steps=this_steps
        self.this_offsets=this_offsets
        self.clip_boxes=clip_boxes
        self.variances=variances
        self.coords=coords
        self.normalize_coords=normalize_coords
        self.n_boxes=n_boxes#!!!!!!!!!!!!!!!!!! Убрать это должно рассчитыаваться автоматически после __init__
    
    #Рассчитаем w и h для каждой bbox ,используя scales & aspect ratio
    
    def call(self,Layer):
        
        size=min(self.img_height,self.img_width)
        
        wh_list=[] #место для хранения w и h каждой из рассчитанных bbox - ов
        
        #вычисляем w & h для каждого aspect ratio
        for ar in self.aspect_ratios:
            
            if (ar == 1):
                
                #Рассчитываем обычные w & h для aspect ratio 1
                box_height=box_width=self.this_scale*size
                
                wh_list.append((box_width,box_height))
                
                if self.two_boxes_for_ar1:
                    
                    #Вычисляем немного более увеличенные версии w & h , используя sqrt , this_scale & next_scale
                    box_height=box_width=np.sqrt(self.this_scale*self.next_scale)*size
                    
                    wh_list.append((box_width,box_height))
                    
            else:
                    
                box_height=self.this_scale*size/np.sqrt(ar) #formula1 for height

                box_width=self.this_scale*np.sqrt(ar)*size #formula2 for width

                wh_list.append((box_width,box_height))
                    
            wh_list=np.array(wh_list)
            
            #we need to get the shape of the Input Tensor
            
            #HERE NUMPY WE NEED TO REPLACE TO TENSORFLOW .get_shape()
            batch_size,feature_map_height,feature_map_width,feature_map_chennles=Layer.shape
            #HERE NUMPY WE NEED TO REPLACE TO TENSORFLOW .get_shape() 
            
            # Compute the grid of box center points. They are identical for all aspect ratios.
            
            # Compute the step sizes, i.e. how far apart the anchor box center points will be vertically and horizontally.
            
            if (self.this_steps is None):
                
                step_height=self.img_height / feature_map_height
                
                step_width=self.img_width / feature_map_width
                
            else:
                
                if isinstance(self.this_steps,(list,tuple)) and (len(self.this_steps)==2):
                    
                    step_height=self.this_step[0]
                    
                    step_width=self.this_steps[1]
                    
                elif isinstance(self.this_steps,(int,float)):
                    
                    step_height=self.this_steps
                    
                    step_width=self.this_steps
                    
            # Compute the offsets, i.e. at what pixel values the first anchor box center point will be from the top and from the left of the image.         
                    
            if (self.this_offsets is None):
                
                offset_height=0.5
                
                offset_width=0.5
                
            else:
                
                if isinstance(self.this_offsets,(list,tuple)) and (len(self.this_offsets)==2):
                    
                    offset_height=self.this_offsets[0]
                    
                    offset_width=self.this_offsets[1]
                    
                elif isinstance(self.this_steps,(int,float)):
                    
                    step_height=self.this_steps
                    
                    step_width=self.this_steps
                    
            
            # Now that we have the offsets and step sizes, compute the grid of anchor box center points.  
            
            cy=np.linspace(offset_height*step_height,(offset_height+feature_map_height -1)*step_height,feature_map_height)
            
            cx=np.linspace(offset_width*step_width,(offset_width+feature_map_width-1)*step_width,feature_map_width)
            
            cx_grid,cy_grid=np.meshgrid(cx,cy)
            
            cx_grid=np.expand_dims(cx_grid, -1) # This is necessary for np.tile() to do what we want further down
            cy_grid=np.expand_dims(cy_grid, -1) # This is necessary for np.tile() to do what we want further down
            
            # Create a 4D tensor template of shape (feature_map_height, feature_map_width, n_boxes, 4)
            # where the last dimension will contain (cx, cy, w, h) 
            
            
            
            boxes_tensor=np.zeros((feature_map_height,feature_map_width,self.n_boxes,4))
            
            #НЕОБХОДИМО В НАЧАЛЕ УСТАНОВИТЬ ЗНАЧЕНИЕ ДЛЯ n_boxes
        
            boxes_tensor[:,:,:,0]=np.tile(cx_grid,(1,1,self.n_boxes)) #Set cx
            boxes_tensor[:, :, :, 1] = np.tile(cy_grid, (1, 1, self.n_boxes)) # Set cy
            boxes_tensor[:, :, :, 2] = wh_list[:, 0] # Set w
            boxes_tensor[:, :, :, 3] = wh_list[:, 1] # Set h
            
            
            # Convert (cx, cy, w, h) to (xmin, xmax, ymin, ymax)
            
            boxes_tensor = convert_coordinates(boxes_tensor, start_index=0, conversion='centroids2corners') 
            
            # If clip_boxes is enabled, clip the coordinates to lie within the image boundaries
            
            if self.clip_boxes:
                
                x_coords=boxes_tensor[:,:,:,[0,2]]
                x_coords[x_coords >= self.imd_width]=self.img_width-1
                x_coords[x_coords<0]=0
                boxes_tensor[:,:,:,[0, 2]] = x_coords
                y_coords = boxes_tensor[:,:,:,[1, 3]]
                y_coords[y_coords >= self.img_height] = self.img_height - 1
                y_coords[y_coords < 0] = 0
                boxes_tensor[:,:,:,[1, 3]] = y_coords
                
            # If normalize_coords is enabled, normalize the coordinates to be within [0,1]
            
            if self.normalize_coords:
                
                boxes_tensor[:,:,:,[0,2]] /=self.img_width
                boxes_tensor[:, :, :, [1, 3]] /= self.img_heigh
                
            # TODO: Implement box limiting directly for `(cx, cy, w, h)` so that we don't have to unnecessarily convert back and forth.
            if self.coords == 'centroids':
                # Convert `(xmin, ymin, xmax, ymax)` back to `(cx, cy, w, h)`.
                boxes_tensor = convert_coordinates(boxes_tensor, start_index=0, conversion='corners2centroids', border_pixels='half')
            elif self.coords == 'minmax':
                # Convert `(xmin, ymin, xmax, ymax)` to `(xmin, xmax, ymin, ymax).
                boxes_tensor = convert_coordinates(boxes_tensor, start_index=0, conversion='corners2minmax', border_pixels='half')
                
          
            # Create a tensor to contain the variances and append it to boxes_tensor. This tensor has the same shape
            # as boxes_tensor and simply contains the same 4 variance values for every position in the last axis.
            
            variances_tensor = np.zeros_like(boxes_tensor) # Has shape `(feature_map_height, feature_map_width, n_boxes, 4)
            variances_tensor += self.variances
            
            
            # Now boxes_tensor becomes a tensor of shape (feature_map_height, feature_map_width, n_boxes, 8)
            boxes_tensor = np.concatenate((boxes_tensor, variances_tensor), axis=-1)
            
            # Now prepend one dimension to boxes_tensor to account for the batch size and tile it along
            # The result will be a 5D tensor of shape (batch_size, feature_map_height, feature_map_width, n_boxes, 8)
            
            boxes_tensor = np.expand_dims(boxes_tensor, axis=0)
            
            boxes_tensor=tf.Variable(initial_value=boxes_tensor,dtype=tf.float32)
            boxes_tensor=tf.reshape(boxes_tensor,[-1,feature_map_height,feature_map_width,self.n_boxes,8])

            return boxes_tensor

In [270]:
def convert_coordinates(tensor,start_index,conversion,border_pixels='half'):

    if border_pixels == 'half':
        d = 0
    elif border_pixels == 'include':
        d = 1
    elif border_pixels == 'exclude':
        d = -1
    
    ind=start_index
    tensor1=np.copy(tensor).astype(np.float)
    
    if conversion == 'minmax2centroids':
        tensor1[..., ind] = (tensor[..., ind] + tensor[..., ind+1]) / 2.0 # Set cx
        tensor1[..., ind+1] = (tensor[..., ind+2] + tensor[..., ind+3]) / 2.0 # Set cy
        tensor1[..., ind+2] = tensor[..., ind+1] - tensor[..., ind] + d # Set w
        tensor1[..., ind+3] = tensor[..., ind+3] - tensor[..., ind+2] + d # Set h
        
    elif conversion=='centroids2minmax':
        tensor1[...,ind]=(tensor[...,ind] - tensor[...,ind+2] / 2.0) # Set xmin
        tensor1[...,ind+1]=(tensor[...,ind] + tensor[...,ind+2] / 2.0) # Set xmax
        tensor1[...,ind+2]=(tensor[...,ind+1] - tensor[...,ind+3] / 2.0) # Set ymin
        tensor1[...,ind+4]=(tensor[...,ind+1] + tensor[...,ind+3] / 2.0) # Set ymax
        
    elif conversion == 'corners2centroids':
        tensor1[..., ind] = (tensor[..., ind] + tensor[..., ind+2]) / 2.0 # Set cx
        tensor1[..., ind+1] = (tensor[..., ind+1] + tensor[..., ind+3]) / 2.0 # Set cy
        tensor1[..., ind+2] = tensor[..., ind+2] - tensor[..., ind] + d # Set w
        tensor1[..., ind+3] = tensor[..., ind+3] - tensor[..., ind+1] + d # Set h
        
    elif conversion == 'centroids2corners':
        tensor1[..., ind] = tensor[..., ind] - tensor[..., ind+2] / 2.0 # Set xmin
        tensor1[..., ind+1] = tensor[..., ind+1] - tensor[..., ind+3] / 2.0 # Set ymin
        tensor1[..., ind+2] = tensor[..., ind] + tensor[..., ind+2] / 2.0 # Set xmax
        tensor1[..., ind+3] = tensor[..., ind+1] + tensor[..., ind+3] / 2.0 # Set ymax
        
    elif (conversion == 'minmax2corners') or (conversion == 'corners2minmax'):
        tensor1[..., ind+1] = tensor[..., ind+2]
        tensor1[..., ind+2] = tensor[..., ind+1]
        
    else:
        raise ValueErrorr("Unexpected conversion value. Supported values are 'minmax2centroids', 'centroids2minmax', 'corners2centroids', 'centroids2corners', 'minmax2corners', and 'corners2minmax'.")
        
    return tensor1

In [244]:
img_height = 300 # Height of the model input images
img_width = 300 # Width of the model input images
img_channels = 3 # Number of color channels of the model input images
mean_color = [123, 117, 104] # The per-channel mean of the images in the dataset. Do not change this value if you're using any of the pre-trained weights.
swap_channels = [2, 1, 0] # The color channel order in the original SSD is BGR, so we'll have the model reverse the color channel order of the input images.
n_classes = 20 # Number of positive classes, e.g. 20 for Pascal VOC, 80 for MS COCO
scales_pascal = [0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05] # The anchor box scaling factors used in the original SSD300 for the Pascal VOC datasets
scales_coco = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05] # The anchor box scaling factors used in the original SSD300 for the MS COCO datasets
scales = scales_pascal
aspect_ratios = [[1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5]] # The anchor box aspect ratios used in the original SSD300; the order matters
two_boxes_for_ar1 = True
steps = [8, 16, 32, 64, 100, 300] # The space between two adjacent anchor box center points for each predictor layer.
offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5] # The offsets of the first anchor box center points from the top and left borders of the image as a fraction of the step size for each predictor layer.
clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are divided as in the original implementation
normalize_coords = True

In [252]:
data=np.arange(0,38*38*512)
Conv_l=data.reshape((-1,38,38,512))

In [274]:
MyBox=AnchorBoxes()

In [276]:
result=MyBox.call(Layer=Conv_l)

In [279]:
result

<tf.Tensor 'Reshape_1:0' shape=(1, 38, 38, 4, 8) dtype=float32>